# Explored combined dataset

- the client observed that weather impacts sales, can the collected data confirm this claim?

In [1]:
CLEANED_SALES_FILE = "./../data/bakery_sales_cleaned.xlsx"
CLEANED_WEATHER_FILE = "./../data/weather_cleaned.xlsx"
PREDICTED_ARTICLE = "TRADITIONAL BAGUETTE"

In [2]:
import pandas as pd
df_sales = pd.read_excel(CLEANED_SALES_FILE)
df_weather = pd.read_excel(CLEANED_WEATHER_FILE)

### Filter sales for target article and create daily summaries

In [3]:
# filter based on target article
df_sales_single_article = df_sales[df_sales['Article']==PREDICTED_ARTICLE]

In [4]:
# aggregate sales volumes to match daily frequency of weather dataset
df_sales_daily_summaries = df_sales_single_article.groupby(df_sales_single_article['datetime'].dt.date)['Quantity'].sum().reset_index()
df_sales_daily_summaries['datetime'] = pd.to_datetime(df_sales_daily_summaries['datetime']) # back to datetime object
df_sales_daily_summaries.head(5)

,datetime,Quantity
0,2021-01-02,128
1,2021-01-03,171
2,2021-01-04,128
3,2021-01-05,99
4,2021-01-07,109


### Combine datasets

In [5]:
# perform a left-join on `datetime` column
df_combined = pd.merge(df_sales_daily_summaries, df_weather, left_on='datetime', right_on='date', how='left')
df_combined['day_of_week'] = df_combined['datetime'].dt.day_name().astype('category') # bring back the day of the week
df_combined.head(5)

,datetime,Quantity,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,day_of_week
0,2021-01-02,128,2021-01-02,1.3,0.5,2.7,0.8,0.0,336.0,24.7,44.0,1010.1,Saturday
1,2021-01-03,171,2021-01-03,0.7,-0.3,1.6,0.0,0.0,327.0,17.1,38.9,1012.1,Sunday
2,2021-01-04,128,2021-01-04,0.0,-1.0,1.1,0.0,0.0,329.0,10.6,30.0,1011.4,Monday
3,2021-01-05,99,2021-01-05,0.8,-0.9,2.1,0.0,0.0,338.0,6.2,30.0,1012.1,Tuesday
4,2021-01-07,109,2021-01-07,0.2,-1.6,2.5,0.0,0.0,343.0,6.2,24.0,1017.1,Thursday


In [6]:
# drop na (2 records we couldn't impute because of missing wind data)
df_combined.dropna(inplace=True)

### Enrich dataset with French holidays

In [7]:
import holidays

france_holidays = holidays.France(years=[2021, 2022])
france_holiday_days = pd.to_datetime(list(france_holidays.keys()))
france_holiday_days

DatetimeIndex(['2021-01-01', '2021-05-01', '2021-05-08', '2021-07-14',
               '2021-11-11', '2021-04-05', '2021-05-24', '2021-05-13',
               '2021-08-15', '2021-11-01', '2021-12-25', '2022-01-01',
               '2022-05-01', '2022-05-08', '2022-07-14', '2022-11-11',
               '2022-04-18', '2022-06-06', '2022-05-26', '2022-08-15',
               '2022-11-01', '2022-12-25'],
              dtype='datetime64[ns]', freq=None)

In [8]:
df_combined['is_holiday'] = df_combined['datetime'].apply(lambda x: x in france_holiday_days)
df_combined.head(5)

,datetime,Quantity,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,day_of_week,is_holiday
0,2021-01-02,128,2021-01-02,1.3,0.5,2.7,0.8,0.0,336.0,24.7,44.0,1010.1,Saturday,False
1,2021-01-03,171,2021-01-03,0.7,-0.3,1.6,0.0,0.0,327.0,17.1,38.9,1012.1,Sunday,False
2,2021-01-04,128,2021-01-04,0.0,-1.0,1.1,0.0,0.0,329.0,10.6,30.0,1011.4,Monday,False
3,2021-01-05,99,2021-01-05,0.8,-0.9,2.1,0.0,0.0,338.0,6.2,30.0,1012.1,Tuesday,False
4,2021-01-07,109,2021-01-07,0.2,-1.6,2.5,0.0,0.0,343.0,6.2,24.0,1017.1,Thursday,False


In [9]:
# data quality check
df_combined[df_combined[['datetime', 'Quantity']].isna().any(axis=1)]

,datetime,Quantity,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,day_of_week,is_holiday


In [10]:
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport
import warnings

# silence Future and User warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="seaborn.matrix")
warnings.filterwarnings("ignore", category=UserWarning, module="ydata_profiling.model.missing")

profile = ProfileReport(df_combined, title="Combined datasets")
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/peter/Documents/osobne/peter/praca/dalej/NICD_Senior_Data_Scientist_jan_24/assessment/preassesment-task/.venv/lib/python3.10/site-packages/ydata_profiling/model/correlations.py:66: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'could not convert string to float: 'Saturday'')
  warnings.warn(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

### Visualisation

In [11]:
# plot daily sales
import plotly.express as px

fig = px.line(df_combined, x='datetime', y='Quantity', title='Selected item sale volumes over time',
              labels={'datetime': 'Date', 'Quantity': 'Quantity Sold'})

fig.show()

In [12]:
# plot sales per week day
# calculate daily sale volumes
sales_per_weekday = df_combined.groupby('day_of_week', observed=False)['Quantity'].mean().reset_index()
sales_per_weekday['Quantity'] = sales_per_weekday['Quantity'].round() 

# ordering of week days
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# plot
fig = px.bar(sales_per_weekday, x='day_of_week', y='Quantity', title=f'Average Sales Volumes per Day: {PREDICTED_ARTICLE}',
             labels={'Quantity': 'Total Quantity Sold', 'day_of_week': 'Day of Week'}, category_orders={'day_of_week': weekday_order})
fig.update_layout(xaxis={'categoryorder': 'array', 'categoryarray': weekday_order})
fig.update_traces(text=sales_per_weekday['Quantity'], textposition='inside')
fig.show()

In [13]:
# plot average sales per holiday and on holiday day
import plotly.express as px
import pandas as pd
import plotly.io as pio

# map colours and encode in the dataframe
colours = {'Regular Day': 'blue', 'Holiday': 'orange'}
df_combined['colour'] = df_combined['is_holiday'].map({True: 'Holiday', False: 'Regular Day'})

# ordering of week days
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Prepare data
avg_sales = df_combined.groupby(['day_of_week', 'colour'], observed=False)['Quantity'].mean().reset_index()
avg_sales['Quantity'] = avg_sales['Quantity'].round().fillna(0).astype(int)  # Round the average sales

# Plot
fig = px.bar(avg_sales, x='day_of_week', y='Quantity', color='colour',
             color_discrete_map=colours, # title=f'Average Sales on Holidays vs Regular Days ({PREDICTED_ARTICLE})',
             labels={'day_of_week': 'Day', 'Quantity': f'Average sales volumes'})
fig.update_layout(barmode='group', legend_title_text='', margin=dict(t=30, b=30, l=20, r=20))
fig.update_layout(xaxis={'categoryorder': 'array', 'categoryarray': weekday_order})

# add the sales volumes (fiddle with colours to match the type of the day)
for i, row in avg_sales.iterrows():
    fig.add_annotation(
        x=row['day_of_week'],
        y=row['Quantity'],
        text=str(row['Quantity']),
        showarrow=False,
        font=dict(color='orange' if row['colour'] == 'Holiday' else 'blue', size=12),
        xanchor='right' if row['colour'] == 'Holiday' else 'left',
        yanchor='bottom'
    )

fig.show()
pio.write_image(fig, './../figures/holiday_vs_regular_day.pdf', height=300)

/Users/peter/Documents/osobne/peter/praca/dalej/NICD_Senior_Data_Scientist_jan_24/assessment/preassesment-task/.venv/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



### Profile combined dataset

In [14]:
from ydata_profiling import ProfileReport

# profile = ProfileReport(df_combined, title="Sales and Weather combined dataset")
# profile.to_widgets()

### Predict the future sales

In [15]:
import pandas as pd
from prophet import Prophet

In [16]:
# Rename columns to fit Prophet's requirements
df_combined = df_combined.rename(columns={'datetime': 'ds', 'Quantity': 'y'})

In [17]:
# one-hot encode week of the day
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df_combined['day_of_week'] = label_encoder.fit_transform(df_combined['day_of_week'])
df_combined['is_holiday'] = label_encoder.fit_transform(df_combined['is_holiday'])
df_combined.head(5)

,ds,y,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,day_of_week,is_holiday,colour
0,2021-01-02,128,2021-01-02,1.3,0.5,2.7,0.8,0.0,336.0,24.7,44.0,1010.1,2,0,Regular Day
1,2021-01-03,171,2021-01-03,0.7,-0.3,1.6,0.0,0.0,327.0,17.1,38.9,1012.1,3,0,Regular Day
2,2021-01-04,128,2021-01-04,0.0,-1.0,1.1,0.0,0.0,329.0,10.6,30.0,1011.4,1,0,Regular Day
3,2021-01-05,99,2021-01-05,0.8,-0.9,2.1,0.0,0.0,338.0,6.2,30.0,1012.1,5,0,Regular Day
4,2021-01-07,109,2021-01-07,0.2,-1.6,2.5,0.0,0.0,343.0,6.2,24.0,1017.1,4,0,Regular Day


In [18]:
# use all available data for training, excluding the last month
df_combined_training = df_combined[df_combined['date']<=pd.Timestamp('2022-08-31 00:00:00')]
df_combined_testing = df_combined[df_combined['date']>pd.Timestamp('2022-08-31 00:00:00')]
print("Number of records for training: ", df_combined_training.shape[0])
print("Number of records for testing: ", df_combined_testing.shape[0])

Number of records for training:  570
Number of records for testing:  28


In [19]:
# Create a Prophet model
model = Prophet(mcmc_samples=1000, interval_width = 0.95)

# Add additional regressors
regressors = ['tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'is_holiday', 'day_of_week']
for regressor in regressors:
    model.add_regressor(regressor)

# Fit the model
model.fit(df_combined_training)

19:37:53 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

19:41:01 - cmdstanpy - INFO - CmdStan done processing.
19:41:01 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: normal_id_glm_lpdf: Scale vector is 0, but must be positive finite! (in 'prophet.stan', line 137, column 2 to line 142, column 4)
	Exception: normal_id_glm_lpdf: Matrix of independent variables is inf, but must be finite! (in 'prophet.stan', line 137, column 2 to line 142, column 4)
Exception: normal_id_glm_lpdf: Scale vector is 0, but must be positive finite! (in 'prophet.stan', line 137, column 2 to line 142, column 4)
	Exception: normal_id_glm_lpdf: Matrix of independent variables is inf, but must be finite! (in 'prophet.stan', line 137, column 2 to line 142, column 4)
Exception: normal_id_glm_lpdf: Scale vector is 0, but must be positive finite! (in 'prophet.stan', line 137, column 2 to line 142, column 4)
	Exception: normal_id_glm_lpdf: Matrix of independent variables is inf, but must be finite! (in 'prophet.stan', line 137, column 2 to line 142, colu

19:41:02 - cmdstanpy - WARNING - Some chains may have failed to converge.
	Chain 1 had 500 iterations at max treedepth (100.0%)
	Chain 2 had 500 iterations at max treedepth (100.0%)
	Chain 3 had 500 iterations at max treedepth (100.0%)
	Chain 4 had 500 iterations at max treedepth (100.0%)
	Use the "diagnose()" method on the CmdStanMCMC object to see further information.


In [20]:
# make prediction on the full dataset
forecast = model.predict(df_combined)

In [21]:
# Calculate error for the test datatest
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

# extract only forecasts for the last month (Sept '22)
testing_forecast = forecast[forecast['ds']>pd.Timestamp('2022-08-31 00:00:00')]['yhat']
rmse = np.sqrt(mean_squared_error(df_combined_testing['y'], np.array(testing_forecast)))
mae = mean_absolute_error(df_combined_testing['y'],  np.array(testing_forecast))
print(f"Root Mean Squared Error: {rmse:.1f}")
print(f"Mean Absolute Error: {mae:.1f}")
print(f"Variance of the sales volumes is {np.var(df_combined['y']):.1f}")

Root Mean Squared Error: 197.2
Mean Absolute Error: 196.7
Variance of the sales volumes is 13702.8


### Use random forest for predictions

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Drop the 'date' column if it is not used as a feature
df_rf = df_combined.drop(['ds', 'colour'], axis=1)


In [25]:
# use all available data for training, excluding the last month
df_rf_training = df_rf[df_rf['date']<=pd.Timestamp('2022-08-31 00:00:00')]
df_rf_testing = df_rf[df_rf['date']>pd.Timestamp('2022-08-31 00:00:00')]
print("Number of records for training: ", df_rf_training.shape[0])
print("Number of records for testing: ", df_rf_testing.shape[0])

Number of records for training:  570
Number of records for testing:  28


In [26]:
X_train = df_rf_training.drop(['y','date'], axis=1)
X_test = df_rf_testing.drop(['y','date'], axis=1)
y_train = df_rf_training['y']
y_test =df_rf_testing['y']

In [27]:
# Define categorical features for one-hot encoding
categorical_features = ['day_of_week', 'is_holiday']

# one-hot encode the day of the week and the holiday info
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features)
    ],
    remainder='passthrough'
)

# create a model pipeline
model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Train the model
model.fit(X_train, y_train)

# Apply the same preprocessing to the test set
X_test_preprocessed = model.named_steps['preprocessor'].transform(X_test)

# Make predictions on the preprocessed test set
y_pred = model.predict(X_test)

# Calculate prediction intervals
prediction_intervals = []

for tree in model.named_steps['regressor'].estimators_:
    tree_predictions = tree.predict(X_test_preprocessed)
    prediction_intervals.append(tree_predictions)

# Calculate the lower and upper bounds of the prediction intervals
lower_bound = np.percentile(prediction_intervals, 2.5, axis=0)
upper_bound = np.percentile(prediction_intervals, 97.5, axis=0)

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print(f'Root Mean Squared Error: {rmse:.2f}')
print(f'Mean Absolute Error: {mae:.2f}')

Root Mean Squared Error: 62.71
Mean Absolute Error: 51.40


### Visualise the predictions

In [28]:
# prophet
df_prophet = forecast[['ds', 'yhat_lower', 'yhat', 'yhat_upper']]
df_prophet.head(5)

,ds,yhat_lower,yhat,yhat_upper
0,2021-01-02,-7.142812,140.248742,292.111733
1,2021-01-03,42.622266,196.138107,346.136352
2,2021-01-04,-64.582689,86.753028,241.259809
3,2021-01-05,-81.164363,65.491577,215.275336
4,2021-01-07,-72.031397,84.002280,239.460239


In [29]:
# make predictions using trained Random Forest
def predict_with_intervals(model, X):
    # Apply the same preprocessing to the input data
    X_preprocessed = model.named_steps['preprocessor'].transform(X)

    # Make predictions
    y_pred = model.predict(X)

    # Calculate prediction intervals
    prediction_intervals = []

    for tree in model.named_steps['regressor'].estimators_:
        tree_predictions = tree.predict(X_preprocessed)
        prediction_intervals.append(tree_predictions)

    # Calculate the lower and upper bounds of the prediction intervals
    lower_bound = np.percentile(prediction_intervals, 2.5, axis=0)
    upper_bound = np.percentile(prediction_intervals, 97.5, axis=0)

    return y_pred, lower_bound, upper_bound

# prep the data for the model predictions
categorical_features = ['day_of_week', 'is_holiday']
X = df_combined.drop(['ds', 'y', 'date', 'colour'], axis=1)  
y = np.array(df_combined['y'])

y_pred, lower_bound, upper_bound = predict_with_intervals(model, X)

# Evaluate the model
mse = mean_squared_error(y, y_pred)
mae = mean_absolute_error(y, y_pred)

print(f'Mean Squared Error: {mse:.2f}')
print(f'Mean Absolute Error: {mae:.2f}')

# wrap relevant data in a new df
df_rf = pd.DataFrame({
    'date': df_combined['date'],
    'Quantity': y,
    'y_pred': y_pred,
    'y_lower': lower_bound,
    'y_upper': upper_bound
})
df_rf.head(5)

Mean Squared Error: 1226.83
Mean Absolute Error: 24.45


,date,Quantity,y_pred,y_lower,y_upper
0,2021-01-02,128,125.77,97.325,215.650
1,2021-01-03,171,169.59,135.325,189.000
2,2021-01-04,128,109.81,70.000,128.000
3,2021-01-05,99,97.13,70.000,116.000
4,2021-01-07,109,102.75,68.950,112.675


In [30]:
# slice the dataframe so the plot is more readable
df_rf_subset = df_rf[df_rf['date']>pd.Timestamp('2022-03-01 00:00:00')]

In [31]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd

# visualise random forest predictions
fig = go.Figure([
    go.Scatter(
        x=df_rf_subset['date'],
        y=df_rf_subset['Quantity'],
        line=dict(color='black', width=1),
        # marker=dict(color='black', size=4), 
        mode='lines',
        name='Quantity'
    ),
    go.Scatter(
        x=df_rf_subset['date'].tolist() + df_rf_subset['date'].tolist()[::-1],
        y=df_rf_subset['y_upper'].tolist() + df_rf_subset['y_lower'].tolist()[::-1],
        fill='toself',
        fillcolor = 'rgba(0, 0, 139, 0.25)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=False,
        name='Prediction Interval'
    ),
    go.Scatter(
        x=df_rf_subset['date'],
        y=df_rf_subset['y_pred'],
        line=dict(color='red', dash='dash', width=1),
        mode='lines',
        name='y_pred'
    )
])

# Customize layout
fig.update_layout(xaxis_title='Date',
                  yaxis_title='Number of Articles',
                  legend_title='Random Forest',
                  margin=dict(t=30, b=30, l=20, r=20),
                  title=f'Random Forest: Predicted vs Actual sales for {PREDICTED_ARTICLE}',
                  yaxis=dict(range=[0, 750])  
)

fig.update_layout(
    legend=dict(
        x=0,
        y=1,
        traceorder='normal',
        font=dict(
            family='Arial',
            size=12,
            color='black'
        ),
        bgcolor='LightSteelBlue',
        bordercolor='Black',
        borderwidth=2
    )
)
                
fig.update_traces(
    name='Actual',
    selector=dict(name='Quantity')
)
fig.update_traces(
    name='Predicted',
    selector=dict(name='y_pred')
)
# show and save the plot
fig.show()
pio.write_image(fig, './../figures/rf_predictions.pdf', height=250)

In [32]:
# get predictions to the prophet dataset
df_prophet = pd.merge(df_prophet, df_rf[['date', 'Quantity']], left_on='ds', right_on='date', how='left')

In [33]:
# slice the dataframe so the plot is more readable
df_prophet_subset = df_prophet[df_prophet['ds']>pd.Timestamp('2022-03-01 00:00:00')]
df_prophet_subset.head(5)

,ds,yhat_lower,yhat,yhat_upper,date,Quantity
389,2022-03-02,-88.371996,68.287904,223.462191,2022-03-02,119
390,2022-03-03,-50.608302,97.859858,254.010854,2022-03-03,122
391,2022-03-04,-52.715906,100.747322,257.684124,2022-03-04,141
392,2022-03-05,-6.895282,155.225781,307.284318,2022-03-05,199
393,2022-03-06,63.870834,213.788012,361.788806,2022-03-06,251


In [34]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# visualise random forest predictions
fig = go.Figure([
    go.Scatter(
        x=df_prophet_subset['date'],
        y=df_prophet_subset['Quantity'],
        line=dict(color='black', width=1),
        # marker=dict(color='black', size=4), 
        mode='lines',
        name='Quantity'
    ),
    go.Scatter(
        x=df_prophet_subset['date'].tolist() + df_prophet_subset['date'].tolist()[::-1],
        y=df_prophet_subset['yhat_upper'].tolist() + df_prophet_subset['yhat_lower'].tolist()[::-1],
        fill='toself',
        fillcolor = 'rgba(0, 0, 139, 0.25)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=False,
        name='Prediction Interval'
    ),
    go.Scatter(
        x=df_prophet_subset['date'],
        y=df_prophet_subset['yhat'],
        line=dict(color='red', dash='dash', width=1),
        mode='lines',
        name='yhat'
    )
])

# Customize layout
fig.update_layout(xaxis_title='Date',
                  yaxis_title='Number of Articles',
                  legend_title='Prophet',
                  margin=dict(t=30, b=30, l=20, r=20),
                  title=f'Prophet: Predicted vs Actual sales for {PREDICTED_ARTICLE}',
                  yaxis=dict(range=[0, 750])  
)

fig.update_layout(
    legend=dict(
        x=0,
        y=1,
        traceorder='normal',
        font=dict(
            family='Arial',
            size=12,
            color='black'
        ),
        bgcolor='LightSteelBlue',
        bordercolor='Black',
        borderwidth=2
    )
)

fig.update_traces(
    name='Actual',
    selector=dict(name='Quantity')
)
fig.update_traces(
    name='Predicted',
    selector=dict(name='yhat')
)

# show and save
fig.show()
pio.write_image(fig, './../figures/prophet_predictions.pdf', height=250)